<a href="https://colab.research.google.com/github/joseph-loeffler/Basketball-Stat-Tracking/blob/main/push-to-master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README

# Project Title: Push to Master List
## Project Overview
This project is designed to transform and integrate financial data from various sources into a unified master sheet. It uses Google Colab, Google Sheets, and Google Drive APIs to authenticate, retrieve, process, and upload data. The data transformation involves mapping property GL codes to their corresponding QuickBooks names, reformatting raw data, and performing necessary data validations before uploading.

## Features
* **Google Sheets Integration**: The project uses Google Sheets to store and manage the data. It authenticates the user and authorizes the Google Sheets API for data access and manipulation.
* **Data Transformation**: The script processes raw data, maps property GL codes to QuickBooks descriptions, reformats data, and performs necessary validations.
* **Error Handling and Retries**: The script includes mechanisms to handle API quota limits and retry operations with exponential backoff.
* **Automated Data Upload**: Processed data is uploaded to the master sheet, with checks to prevent duplicates and ensure data integrity.

## Setup and Requirements
* **Google Colab**: The script is designed to run in Google Colab. Ensure you have access to a Google Colab environment.

## Usage

### Raw Source Files
* **Ramp**
    1. Go to ["Ramp.com/home" > "Accounting" > "Ramp Card"](https://app.ramp.com/accounting/ramp-card/needs-review).
    2. Click "Sync All".
    3. Click "[name of button]".
* **Ramp Reimbursements**
    1. Go to ["Ramp.com/home" > "Accounting" > "Reimbursements"](https://app.ramp.com/accounting/reimbursements/needs-review).
    2. Click "Sync All".
    3. Click "[name of button]".
* **Old Lowe's *(Obsolete)***
    1. Website not working rn...
* **New Lowe's**
    1. TBD.
* **Amazon**
    1. Go to ["Amazon.com" > "Your Account" > "Business analytics" > "Reports"](https://www.amazon.com/b2b/aba/reports?ref=hpr_redirect_your_reports).
    2. Click on "Create report" and select "Orders" from the dropdown menu.
    3. Click on "Adjust columns", and check the box next to all the column categories.
    4. Click "Submit".
    5. Click on the dropdown menu under "Time period" and select "Custom Range". Then, enter your desired start and end dates in their respective input boxes.
    6. Click "Generate report".
* **Delta Amex**
    1. TBD.
* **QLR**
    1. In Google Drive, go to [Property Billback Invoices/Raw Source Files/Time/Marketing Bill Back.gsheet](https://docs.google.com/spreadsheets/d/1vkrNdk9OYIyH5KuVK01CCOvzn7KQhJEkwJ6b2Q23wGA/edit?usp=sharing).
    2. Ensure that you are on the "QLR" sheet.
    3. Click "File" > "Download" > "Comma Separated Values (.csv)"
* **Digital Marketing**
    1. In Google Drive, go to [Property Billback Invoices/Raw Source Files/Time/Marketing Bill Back.gsheet](https://docs.google.com/spreadsheets/d/1vkrNdk9OYIyH5KuVK01CCOvzn7KQhJEkwJ6b2Q23wGA/edit?gid=501220770#gid=501220770).
    2. Ensure that you are on the "Digital Marketing" sheet.
    3. Click "File" > "Download" > "Comma Separated Values (.csv)"
* **JSQ**
    1. In Google Drive, go to [Property Billback Invoices/Raw Source Files/JSQ/JSQ.gsheet](https://docs.google.com/spreadsheets/d/1-0BsuBs__ql-5c49pzgCuD19kqwDGDKye-kNl36GDIQ/edit?usp=drive_link).
    2. Ensure that you are on the "2024" sheet.
    3. Click "File" > "Download" > "Comma Separated Values (.csv)"
* **Jeremy**
    1. In Google Drive, go to [Property Billback Invoices/Raw Source Files/Jeremy/Sholler Time Billing.gsheet](https://docs.google.com/spreadsheets/d/1RQdivyTOo55ybzBVd62ASzuDOAT4ht9cbGWHHCE7OA4/edit?usp=drive_link).
    2. Ensure that you are on the "Sheet1" sheet.
    3. Click "File" > "Download" > "Comma Separated Values (.csv)"


## Maintaining this Project
* **Constants and global vars**: Make sure to update any constant values defined after the imports. This includes the column-name dictionaries, the property-names list, folder and file links, etc.
    * Rarely, there are constants that are defined in functions, such as the SaasAnt mandatory columns
* **When 2025 comes around**: The script is coded to work with the 2024 versions of many of the live sheets. The LIVE_SHEET_DATE_CUTOFF constant and the urls/sheet names to those live sheets will need to be updated eventually.

## Notes
* **"spread" vs. "sheet"**: I use "spread" to denote Google Sheets files, which may or may not include more than one worksheet. I use "sheet" to denote individual worksheets.

## Conclusion
This project streamlines the process of transforming and integrating financial data from multiple sources into a unified master sheet, leveraging Google Colab and Google APIs for efficient data management and processing.

In [ ]:
# Setup gspread
from google.colab import auth
from google.auth import default
import gspread

# Library imports
from datetime import datetime
from functools import wraps
from dateutil import parser
from typing import List, Set
import pandas as pd
import random
import time
import os
import re

# Setup Google Drive API
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


# Authenticate user
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Build Drive service
drive_service = build('drive', 'v3', credentials=creds)


# Mapping from Property GL Codes to their QuickBooks names
GL_CODE_TO_QB_DICT = {
    'N/A': 'N/A',
    'n/a': 'N/A',
    'NA': 'N/A',
    'na': 'N/A',
    '1510-020': 'Entity Expenses:Financing Fee',
    '1910-800': 'Entity Expenses:Acquisition Fee',
    '2110-500': 'Entity Expenses:A/P - OTHER',
    '6210-000': 'Marketing:Advertising',
    '6210-050': 'Marketing:Marketing Software Fees',
    '6210-100': 'Marketing:PPC Advertising',
    '6210-105': 'Marketing:Community Outreach/Events',
    '6220-000': 'Marketing:Locator/Broker Fees',
    '6220-005': 'Marketing:Resident Referral Fees',
    '6250-010': 'Marketing:Resident Events/Retention',
    '6250-020': 'Marketing:Other Marketing',
    '6250-040': 'Marketing:Reputation Management',
    '6250-100': 'Administrative:Prospect Credit Check Fees',
    '6250-200': 'Administrative:Furniture Rental/Storage',
    '6250-225': 'Administrative:Package Locker',
    '6250-600': 'Marketing:Leasing Promotions/Incentives',
    '6250-700': 'Administrative:Dues/Subscriptions',
    '6310-500': 'Administrative:Office Equipment',
    '6311-000': 'Administrative:Office Supplies',
    '6320-000': 'Fixed Expenses:Property Management Fees',
    '6340-000': 'Administrative:Legal Fees',
    '6340-010': 'Administrative:Professional/Consulting Fees',
    '6360-050': 'Administrative:Answering Service Fees',
    '6360-100': 'Administrative:Office/Recreation Telecom Fees',
    '6360-400': 'Administrative:Cell Phone',
    '6370-100': 'Administrative:Postage and Mail',
    '6370-200': 'Administrative:Bank Fees/Interest Expense',
    '6370-300': 'Administrative:Travel/Mileage',
    '6370-500': 'Administrative:Meals & Entertainment',
    '6390-000': 'Administrative:Other Admin.',
    '6390-010': "Administrative:Renter's Insurance",
    '6390-065': 'Administrative:Software Subscription Fees',
    '6390-100': 'Fixed Expenses:HOA Dues',
    '6390-150': 'Administrative:Training & Education',
    '6390-160': 'Administrative:Secret Shop Surveys',
    '6515-000': 'Repairs & Maintenance:Cleaning',
    '6519-000': 'Repairs & Maintenance:Pest Control Service',
    '6520-000': 'Marketing:Signage/Brochures',
    '6536-500': 'Repairs & Maintenance:Landscaping/Irrigation',
    '6541-050': 'Repairs & Maintenance:Appliances',
    '6541-200': 'Repairs & Maintenance:Electrical',
    '6541-300': 'Repairs & Maintenance:Plumbing',
    '6541-400': 'Repairs & Maintenance:Flooring',
    '6541-600': 'Repairs & Maintenance:Lightning',
    '6541-904': 'Repairs & Maintenance:Windows & Doors',
    '6541-907': 'Repairs & Maintenance:Roof and Gutter',
    '6541-908': 'Administrative:Employee Uniforms',
    '6542-901': 'Repairs & Maintenance:Fire Protection',
    '6542-909': 'Repairs & Maintenance:Gutter Cleaning',
    '6542-910': 'Repairs & Maintenance:Carpentry and Finish Work',
    '6542-911': 'Repairs & Maintenance:Security/Camera Systems',
    '6543-900': 'Repairs & Maintenance:Dryer Vent/Chimney Cleaning',
    '6545-050': 'Repairs & Maintenance:Elevator Repairs/Service Calls',
    '6546-100': 'Repairs & Maintenance:HVAC',
    '6547-050': 'Repairs & Maintenance:Pool',
    '6560-110': 'Repairs & Maintenance:Painting',
    '6561-100': 'Repairs & Maintenance:Window Coverings',
    '6570-000': 'Repairs & Maintenance:Vehicle Repairs and Maintenance',
    '6580-000': 'Repairs & Maintenance:Gates & Fences',
    '6580-200': 'Repairs & Maintenance:Recreational Facilities',
    '6581-000': 'Repairs & Maintenance:Garage/Parking',
    '6590-000': 'Repairs & Maintenance:Other R&M',
    '6619-000': 'Turnover:Pest Control',
    '6642-908': 'Turnover:Tubs/Showers Repairs',
    '6642-909': 'Repairs & Maintenance:Cabinet/Countertop Repairs',
    '6647-000': 'Turnover:Cleaning',
    '6647-100': 'Turnover:Carpet Cleaning',
    '6660-100': 'Turnover:Touch-Up Painting',
    '6690-000': 'Turnover:Trash Removal',
    '6690-100': 'Turnover:Other Turnover',
    '6710-000': 'Fixed Expenses:Real Estate Property Taxes',
    '6710-020': 'Fixed Expenses:Property Tax - Other',
    '6719-000': 'Fixed Expenses:Business Tax & License',
    '6719-010': 'Fixed Expenses:Ground Lease',
    '6719-100': 'Fixed Expenses:Personal Property Taxes',
    '6720-000': 'Fixed Expenses:Property Insurance',
    '6728-000': 'Fixed Expenses:Vehicle Insurance & Registration',
    '7120-000': 'Entity Expenses:Legal Fees',
    '7120-005': 'Entity Expenses:Marketing Fees',
    '7120-015': 'Entity Expenses:Professional/Consulting Fees',
    '7120-045': 'Professional Fees:Accounting/Tax Prep Fees',
    '7120-200': 'Entity Expenses:Asset management fee',
    '7190-000': 'Entity Expenses:Other Partnership Expenses',
    '7190-003': 'Entity Expenses:Exec. Board Related',
    '7190-020': 'Entity Expenses:Insurance Related',
    '7190-400': 'Entity Expenses:Acquisition Costs',
    '7290-900': 'Entity Expenses:Other Pre-Opening Expenses',
    '7310-010': 'Non Unit Renovation:Concrete/Walkways',
    '7310-100': 'Replacement:Building Exteriors',
    '7310-205': 'Replacement:Elevators',
    '7310-210': 'Replacement:Mirrors/Medicine Cabinets',
    '7310-240': 'Replacement:Electrical Infrastructure',
    '7310-250': 'Replacement:Office Equipment',
    '7310-400': 'Replacement:Plumbing Fixtures',
    '7315-020': 'Replacement:Tree Trimming',
    '7320-208': 'Replacement:Concrete/Walkways',
    '7320-300': 'Replacement:Exterior Lighting',
    '7320-400': 'Replacement:Gates & Fences',
    '7320-500': 'Replacement:Pool',
    '7320-800': 'Replacement:Maintenance Equipment / Shop',
    '7320-807': 'Replacement:Fire Protection',
    '7325-000': 'Replacement:Recreation & Amenity Areas',
    '7330-010': 'Replacement:Garages/Carports',
    '7335-100': 'Replacement:Security Systems/Cameras',
    '7338-000': 'Replacement:Solar',
    '7355-100': 'Non Unit Renovation:Security Systems/Cameras',
    '7362-000': 'Unit Renovation:Corporate Furniture',
    '7365-010': 'Replacement:Golf Cart',
    '7390-000': 'Non Unit Renovation:Access Control',
    '7390-350': 'Non Unit Renovation:EV Charging',
    '7391-900': 'Non Unit Renovation:Capital Maintenance Payroll',
    '7502-023': 'Unit Renovation:Oversight/Mgmt. Fee',
    '7505-000': 'Unit Renovation:Interior Doors/Closet Doors',
    '7515-000': 'Unit Renovation:Painting Supplies & Labor',
    '7520-000': 'Unit Renovation:Cabinets',
    '7530-000': 'Unit Renovation:Interior Plumbing Fixtures',
    '7538-000': 'Non Unit Renovation:Solar',
    '7540-000': 'Unit Renovation:Countertops',
    '7550-000': 'Unit Renovation:Tub/Shower Replacement',
    '7556-000': 'Unit Renovation:Washers & Dryers',
    '7565-000': 'Unit Renovation:Mirrors/Medicine Cabinets',
    '7580-000': 'Unit Renovation:Carpets',
    '7582-000': 'Unit Renovation:Vinyl/Hard Surface Flooring',
    '7585-100': 'Unit Renovation:Dishwashers',
    '7585-200': 'Unit Renovation:Stoves/Ranges',
    '7585-300': 'Unit Renovation:Refrigerators',
    '7585-400': 'Unit Renovation:Garbage Disposals',
    '7590-000': 'Unit Renovation:Window Coverings',
    '7591-000': 'Unit Renovation:Smart Home',
    '7591-400': 'Unit Renovation:Interior Electrical Fixtures',
    '7595-000': 'Unit Renovation:Other Interior Reno',
    '7596-000': 'Unit Renovation:Permits/Fees',
    '7597-000': 'Unit Renovation:Interior Framing/Drywall',
    '7599-010': 'Non Unit Renovation:Rebates/Incentives',
    '7605-000': 'Non Unit Renovation:New Construction',
    '7615-000': 'Non Unit Renovation:Common Area Laundry',
    '7615-100': 'Non Unit Renovation:Building Lobbies/Entryways',
    '7625-100': 'Non Unit Renovation:Maintenance Equipment / Shop',
    '7630-000': 'Non Unit Renovation:Asphalt/Parking Lot',
    '7634-000': 'Non Unit Renovation:Office/Clubhouse',
    '7634-200': 'Non Unit Renovation:Fitness Center',
    '7635-000': 'Non Unit Renovation:Landscaping/Irrigation',
    '7640-000': 'Non Unit Renovation:Exterior Painting',
    '7641-000': 'Non Unit Renovation:HVAC',
    '7643-000': 'Non Unit Renovation:Model Furniture',
    '7655-000': 'Non Unit Renovation:Pool & Furniture',
    '7655-200': 'Non Unit Renovation:Outdoor Bbq Area',
    '7655-300': 'Non Unit Renovation:Sports Court',
    '7655-500': 'Non Unit Renovation:Elevators',
    '7655-750': 'Non Unit Renovation:Marketing',
    '7655-900': 'Non Unit Renovation:Pet Wash Station',
    '7660-000': 'Non Unit Renovation:Roofs',
    '7661-000': 'Non Unit Renovation:Building Interior Hallways',
    '7665-000': 'Non Unit Renovation:Signage',
    '7665-800': 'Non Unit Renovation:Dog Park',
    '7670-000': 'Non Unit Renovation:Exterior Siding',
    '7675-000': 'Non Unit Renovation:Gates & Fences',
    '7677-000': 'Non Unit Renovation:Green Improvements',
    '7680-000': 'Non Unit Renovation:Playground',
    '7685-000': 'Non Unit Renovation:Garages/Carports',
    '7690-000': 'Non Unit Renovation:Oversight/Mgmt. Fee',
    '7690-100': 'Non Unit Renovation:Design/Consulting/Professional Fees',
    '7710-000': 'Replacement:Garbage Disposals',
    '7710-100': 'Replacement:Dishwashers',
    '7710-200': 'Replacement:Stoves/Ranges',
    '7710-300': 'Replacement:Refrigerators',
    '7710-400': 'Replacement:Washer & Dryers',
    '7710-500': 'Replacement:Water Heaters',
    '7715-000': 'Replacement:Cabinets/Countertops',
    '7720-000': 'Replacement:Carpets',
    '7725-000': 'Replacement:Vinyl/Hard Surface Flooring',
    '7730-000': 'Replacement:Window Coverings',
    '7740-000': 'Replacement:Interior Electrical Fixtures',
    '7745-000': 'Replacement:HVAC',
    '7764-000': 'Replacement:Signage',
    '7765-000': 'Replacement:Landscaping & Irrigation',
    '7766-000': 'Replacement:Asphalt/Parking Lot',
    '7770-000': 'Replacement:Mail Boxes/Package Lockers',
    '7775-000': 'Replacement:Other Replacement',
    '7780-000': 'Replacement:Pest Control',
    '7781-000': 'Replacement:Plumbing Infrastructure',
    '7782-000': 'Replacement:Tubs/Showers',
    '7783-000': 'Replacement:Gutters',
    '7784-000': 'Replacement:Roof',
    '7785-000': 'Replacement:Interior Painting',
    '7785-100': 'Replacement:Carpentry and Finish Work',
    '7786-000': 'Replacement:Windows/Doors',
    '7786-500': 'Replacement:Keys & Access Control',
    '7790-000': 'Replacement:Insurance Reimbursement',
    '8212-015': 'Replacement:EV Charging',
    '8212-035': 'Non Unit Renovation:Windows',
    '8214-020': 'Non Unit Renovation:Exterior Doors',
    '8216-050': 'Non Unit Renovation:Boilers/Water Heaters',
    '8218-040': 'Non Unit Renovation:Plumbing Infrastructure',
    '8218-044': 'Non Unit Renovation:Electrical Infrastructure',
    '8220-010': 'Non Unit Renovation:Other Amenity/Recreation Area',
    '8220-025': 'Non Unit Renovation:Exterior Lighting',
    '8220-042': 'Non Unit Renovation:Drainage',
    '8254-030': 'Marketing:Resident Move-In Gifts'}

# Variable names used in code -> source column names
col_names = {
    'property name': {
        'Master': 'Property Name',
        'Ramp': 'QuickBooks Customer/Job',
        'RampR': 'QuickBooks Customer/Job',
        'Amazon': 'Custom Field 2',
        'OldLowes': 'Property Name',
        'QLR': 'Property Name',
        'DigitalMarketing': 'Property Name',
        'JSQ': 'Property Name',
        'Jeremy': 'Property Name',
        'DeltaAmex': 'Property Name'
    },
    'source':{
        'Master': 'Source'
    },
    'GL code': {
        'Master': 'Property GL Code',
        'Ramp': 'Property GL Code',
        'RampR': 'Property GL Code',
        'Amazon': 'Property GL Code',
        'OldLowes': 'Property GL Code',
        'QLR': 'Property GL Code',
        'DigitalMarketing': 'Property GL Code',
        'JSQ': 'Property GL Code',
        'Jeremy': 'Property GL Code',
        'DeltaAmex': 'Property GL Code'
    },
    'GL code description': {
        'Master': 'Property GL Code Description',
        'Ramp': 'Property GL Code Description',
        'RampR': 'Property GL Code Description',
        'Amazon': 'Property GL Code Description',
        'OldLowes': 'Property GL Code Description',
        'QLR': 'Property GL Code Description',
        'DigitalMarketing': 'Property GL Code Description',
        'JSQ': 'Property GL Code Description',
        'Jeremy': 'Property GL Code Description',
        'DeltaAmex': 'Property GL Code Description'
    },
    'QB GL code description':{
        'Master': 'Property GL Code Description (QuickBooks)'
    },
    'AI GL code': {
        'Master': '(AI Predicted) Property GL Code',
        'Ramp': '(AI Predicted) Property GL Code',
        'RampR': '(AI Predicted) Property GL Code',
        'Amazon': '(AI Predicted) Property GL Code',
        'OldLowes': '(AI Predicted) Property GL Code',
        'DeltaAmex': '(AI Predicted) Property GL Code'
    },
    'transaction date': {
        'Master': 'Transaction Date',
        'Ramp': 'Transaction Date',
        'RampR': 'Transaction Date',
        'Amazon': 'Order Date',
        'OldLowes': 'Payment Date',
        'QLR': 'Date',
        'DigitalMarketing': 'Date',
        'JSQ': 'Date',
        'Jeremy': 'Date',
        'DeltaAmex': 'Date'
    },
    'corporate account': {
        'Master': 'Accounting Category',
        'Ramp': 'QuickBooks Category',
        'RampR': 'QuickBooks Category'
    },
    'transaction description': {
        'Master': 'Item Name/Description',
        'Amazon': 'Title',
        'OldLowes': 'Expense Description',
        'Jeremy': 'Project',
        'DeltaAmex': 'Description'
    },
    'memo': {
        'Master': 'Memo',
        'Ramp': 'Memo',
        'RampR': 'Memo',
        'Amazon': 'Custom Field 4',
        'OldLowes': 'Memo'
    },
    'debit': {
        'Master': 'Debit',
        'Ramp': 'Debit',
        'RampR': 'Debit',
        'Amazon': 'Item Net Total',
        'OldLowes': 'Expense Line Amount',
        'QLR': 'Amount',
        'DigitalMarketing': 'Amount',
        'JSQ': 'Amount',
        'Jeremy': 'Amount',
        'DeltaAmex': 'Amount'
    },
    'credit': {
        'Master': 'Credit',
        'Ramp': 'Credit',
        'RampR': 'Credit'
    },
    'vendor': {
        'Master': 'Vendor',
        'Ramp': 'Merchant Name',
        'RampR': 'Merchant Name'
    },
    'user': {
        'Master': 'User',
        'Ramp': 'Line Memo',
        'RampR': 'Requester Name',
        'Amazon': 'Account User',
        'DeltaAmex': 'Card Member'
    },
    'SKU': {
        'Master': 'SKU/ASIN',
        'Amazon': 'ASIN',
        'OldLowes': 'Expense Description'
    },
    'uploaded to saasant as expense?': {
        'Master': 'Uploaded to SaasAnt as Expense?'
    },
    'uploaded to saasant as invoice?': {
        'Master': 'Uploaded to SaasAnt as Invoice?'
    },
    'uploaded to saasant as bill?': {
        'Master': 'Uploaded to SaasAnt as Bill?'
    },
    'unique id': {
        'Master': 'Unique ID (from source)',
        'Ramp': 'External ID',
        'RampR': 'Reimbursement Link',
        'Amazon': 'Order ID',
        'OldLowes': 'Ref No',
        'DeltaAmex': 'Reference'
    },
    'transaction link': {
        'Master': 'Transaction Link',
        'Ramp': 'Transaction Link',
        'RampR': 'Reimbursement Link'
    },
    'Ramp category': {
        'Master': 'Ramp Category',
        'Ramp': 'Ramp Category'
    },
    'Amazon category': {
        'Master': 'Amazon Category',
        'Amazon': 'Commodity'
    },
    'DeltaAmex category': {
        'Master': 'DeltaAmex Category',
        'DeltaAmex': 'Category'
    },
    'Amazon payment ref id': {
        'Master': 'Amazon Payment Reference ID',
        'Amazon': 'Payment Reference ID'
    },
    'batch link': {
        'Master': 'Link to all transactions'
    },
    'date pushed': {
        'Master': 'Date Pushed to Master Sheet'
    }}

def build_dict(col_names):
    """
    Builds a dictionary that maps source names to dictionaries of attribute-column name pairs.

    Example:
        Input:
            col_names = {
                'property name': {
                    'Master': 'Property Name',
                    'Ramp': 'QuickBooks Customer/Job'
                },
                'GL code': {
                    'Master': 'Property GL Code',
                    'Ramp': 'Property GL Code'
                }
            }

        Output:
            {
                'Master': {
                    'property name': 'Property Name',
                    'GL code': 'Property GL Code'
                },
                'Ramp': {
                    'property name': 'QuickBooks Customer/Job',
                    'GL code': 'Property GL Code'
                }
            }
    """
    source_specific_dict = {}
    for attribute, sources in col_names.items():
        for source, col_name in sources.items():
            if source not in source_specific_dict:
                source_specific_dict[source] = {}
            source_specific_dict[source][attribute] = col_name

    return source_specific_dict

# Build the dictionary
COL_NAMES = build_dict(col_names)

READY_TO_UPLOAD_TO_MASTER_SHEET_FOLDER_URL = (
    'https://drive.google.com/drive/folders/1Cchn82SQmZjtnsfUEoV3nXMldUtQoCOX')
ARCHIVED_CODED_FILES_FOLDER_URL = (
    'https://drive.google.com/drive/folders/1xFDMmzQax-Y2t8LHgUJ_7-XvY_wEuUHW')

CORPORATE_PROPERTY_COA_MAPPING_SHEET = {
    'url': ('https://docs.google.com/spreadsheets/d/1uc3oDxme39zJj9C-R92YohgV4l--MxesDaRz8bUvQo4/ed'
            'it?gid=0#gid=0'),
    'sheet title': 'Mapping'}

MASTER_SHEET = {
    'url': ('https://docs.google.com/spreadsheets/d/15nqM-3Jo-4t_JM7u_shRiBjXIIJ9JhokPyO5f6d1KoY/ed'
            'it?gid=0#gid=0'),
    'sheet title': 'Master List'}

NON_BILLABLE_SHEET = {
    'url': ('https://docs.google.com/spreadsheets/d/1XtrjNXQ8Spb0n6qzIubY3g7OeJ51CJxlm-1VZD8eReQ/ed'
            'it?gid=0#gid=0'),
    'sheet title': 'Non-Billable Transactions'}

LIVE_SHEETS = {
    'QLR': {
        'url': ('https://docs.google.com/spreadsheets/d/1vkrNdk9OYIyH5KuVK01CCOvzn7KQhJEkwJ6b2Q23wG'
                'A/edit?gid=0#gid=0'),
        'sheet title': 'QLR',
        'header row idx': 3,  # Google Sheets index (so 1-indexed)
        'GL code': '6250-020',
        'GL code description': 'Other Marketing'
    },
    'DigitalMarketing': {
        'url': ('https://docs.google.com/spreadsheets/d/1vkrNdk9OYIyH5KuVK01CCOvzn7KQhJEkwJ6b2Q23wG'
                'A/edit?gid=0#gid=0'),
        'sheet title': 'DigitalMarketing',
        'header row idx': 3,  # Google Sheets index (so 1-indexed)
        'GL code': '6210-100',
        'GL code description': 'PPC Advertising'
    },
    'JSQ': {
        'url': ('https://docs.google.com/spreadsheets/d/1-0BsuBs__ql-5c49pzgCuD19kqwDGDKye-kNl36GDI'
                'Q/edit?gid=969030513#gid=969030513'),
        'sheet title': '2024',
        'header row idx': 3,  # Google Sheets index (so 1-indexed)
        'GL code': '7190-000',
        'GL code description': 'Other Partnership Expenses'
    },
    'Jeremy': {
        'url': ('https://docs.google.com/spreadsheets/d/1RQdivyTOo55ybzBVd62ASzuDOAT4ht9cbGWHHCE7OA'
                '4/edit?gid=0#gid=0'),
        'sheet title': 'Sheet1',
        'header row idx': 1,  # Google Sheets index (so 1-indexed)
        'GL code': pd.NA,
        'GL code description': pd.NA
    }}

JEREMY_HOURLY_RATE = 100
LIVE_SHEET_DATE_CUTOFF = '2024-01-01'
RELOCATE_SPREAD_MODE = False
MAX_API_RETRIES = 8  # Waits up to 128 seconds (for a total of 255 seconds)

# Add property names after acquisitions
PROPERTY_NAMES = [
    'Battle Ground',            #1
    'Briggs Village',           #2
    'Clock Tower Village',      #3
    'Crossings at Chapel Hill', #4
    'Fern Ridge',               #5
    'Lakeview Commons',         #6
    'Laurel Creek',             #7
    'Salish Flats',             #8
    'Towncenter',               #9
    'Township Canby',           #10
    'Township Eastside',        #11
    'Township Lake Meridian',   #12
    'Township Sherwood',        #13
    'Village'                   #14
    #'Meadowscape'               #15
    ]


# General Google Drive API functions to load spreadsheets as dataframes,
#  move files, get files info from folder, etc.
def retry_on_quota_exceeded():
    """
    Decorator to retry a function on gspread API quota exceeded errors (HTTP 429).
    Retries the function with exponential backoff up to MAX_API_RETRIES times.

    Returns:
        function: The decorated function with retry logic.
    """
    def decorator_retry(func):
        @wraps(func)
        def wrapper_retry(*args, **kwargs):
            retry_count = 0
            while retry_count < MAX_API_RETRIES:
                try:
                    return func(*args, **kwargs)
                except gspread.exceptions.APIError as e:
                    error = e.response.json()
                    if error['error']['code'] == 429:
                        # Handle quota exceeded error with exponential backoff
                        retry_count += 1
                        sleep_time = (2 ** retry_count) + (random.uniform(0, 1))
                        print(f"API quota exceeded, retrying in {sleep_time:.2f} seconds...")
                        time.sleep(sleep_time)
                    else:
                        raise  # Re-raise the exception if it's not a quota exceeded error
            raise Exception("Max retries exceeded. Could not complete the request.")
        return wrapper_retry
    return decorator_retry


@retry_on_quota_exceeded()
def open_sheet_from_url(spread_url, sheet_index=0, sheet_title=None):
    """Opens a Google Sheet by URL, returning a specific worksheet."""
    if sheet_title:
        sheet = gc.open_by_url(spread_url).worksheet(sheet_title)
    else:
        sheet = gc.open_by_url(spread_url).get_worksheet(sheet_index)
    return sheet


@retry_on_quota_exceeded()
def df_from_spread_url(spread_url, sheet_index=0, sheet_title=None):
    """Converts a Google Sheet to a pandas DataFrame."""
    sheet = open_sheet_from_url(spread_url, sheet_index, sheet_title)
    return pd.DataFrame(sheet.get_all_records())


@retry_on_quota_exceeded()
def df_from_csv_url(url, header_row_idx=None, index_col_idx=None):
    """Converts a CSV file from a Google Drive URL to a pandas DataFrame."""
    file_id = extract_drive_id(url)
    request = drive_service.files().get_media(fileId=file_id)
    file_path = f'/tmp/{file_id}.csv'
    with open(file_path, 'wb') as f:
        downloader = MediaIoBaseDownload(f, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
    return pd.read_csv(file_path, header=header_row_idx, index_col=index_col_idx)


def df_from_file_url(url, filetype):
    """Converts a csv or Google Sheets from a Google Drive URL to a pandas DataFrame."""
    if filetype == 'text/csv':
        df = df_from_csv_url(url)
    elif filetype == 'application/vnd.google-apps.spreadsheet':
        df = df_from_spread_url(url)
    else:
        raise ValueError(f"Unknown file type: {filetype}")
    return df


@retry_on_quota_exceeded()
def get_cols_by_name_from_sheet(sheet, column_names, header_row=1) -> pd.DataFrame:
    """
    Get multiple columns in a pd.DataFrame from a Google Sheet based on the column names.

    Parameters:
        sheet (gspread.Sheet): The Google Sheet object.
        column_names (list): A list of column names to fetch.
        header_row (int): The row number of the header (1-indexed).

    Returns:
        pd.DataFrame: A DataFrame containing the columns.
        None: If any column is not found.
    """
    # Get the header row
    header = sheet.row_values(header_row)

    # Find the indices of the columns
    col_indices = []
    for column_name in column_names:
        try:
            col_idx = header.index(column_name) + 1
            col_indices.append((column_name, col_idx))
        except ValueError:
            raise ValueError(f"Column '{column_name}' not found in the sheet.")

    # Get all values in the specified columns, including the header
    data = {}
    for col_name, col_idx in col_indices:
        column_values = sheet.col_values(col_idx)
        data[col_name] = column_values[1:]  # Skip the header

    # Create a DataFrame
    df = pd.DataFrame(data)

    return df


def extract_drive_id(url):
    """Extracts the Google Drive file ID from various types of URLs."""
    # Define a regular expression pattern to match the ID in various types of Google Drive URLs
    pattern = r'(?:drive/(?:folders|file|d)/|docs/(?:spreadsheets/d/))([a-zA-Z0-9-_]+)'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        # Another attempt to match "spreadsheets" and "file" type URLs specifically
        pattern_spreadsheets = r'/spreadsheets/d/([a-zA-Z0-9-_]+)'
        match_spreadsheets = re.search(pattern_spreadsheets, url)
        if match_spreadsheets:
            return match_spreadsheets.group(1)

        pattern_file = r'/file/d/([a-zA-Z0-9-_]+)'
        match_file = re.search(pattern_file, url)
        if match_file:
            return match_file.group(1)

        return None


@retry_on_quota_exceeded()
def get_files_info_from_folder(folder_url):
    """
    Retrieves information about files in a Google Drive folder.

    Args:
        folder_url (str): The URL of the Google Drive folder.

    Returns:
        List[dict]: A list of dictionaries containing file information,
                    each with 'name', 'url', and 'type' keys.
    """
    folder_id = extract_drive_id(folder_url)
    query = f"'{folder_id}' in parents and trashed=false"
    results = drive_service.files().list(
        q=query,
        fields="files(id, name, webViewLink, mimeType)"
    ).execute()
    items = results.get('files', [])

    files_info = []
    for item in items:
        files_info.append({
            'name': item['name'],
            'url': item['webViewLink'],
            'type': item['mimeType']
        })
    return files_info


@retry_on_quota_exceeded()
def move_file(file_url, dst_folder_url):
    """
    Moves a Google Drive file to a specified folder.

    Args:
        file_url (str): The URL of the Google Drive file.
        dst_folder_url (str): The URL of the destination Google Drive folder.
    """
    # Extract folder ID from destination folder URL
    dst_folder_id = extract_drive_id(dst_folder_url)

    # Extract file ID from file URL
    file_id = extract_drive_id(file_url)

    # Retrieve the current parents to remove them
    file = drive_service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))

    # Move the file to the new folder
    drive_service.files().update(
        fileId=file_id,
        addParents=dst_folder_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()


@retry_on_quota_exceeded()
def append_df_to_sheet(df, dst_sheet):
    """Appends pandas DataFrame to Google Sheet"""
    # Get the column names of the destination sheet
    sheet_cols = dst_sheet.row_values(1)
    df_cols = list(df.columns)

    # Only keep columns in df that exist in the destination sheet
    non_dst_cols = [col for col in df_cols if col not in sheet_cols]
    df = df.drop(columns=non_dst_cols)

    # Reindex df to match the order of columns in the destination sheet
    df = df.reindex(columns=sheet_cols)

    # Replace null values with an empty string
    df.fillna('', inplace=True)

    # Append rows to the destination sheet
    dst_sheet.append_rows(df.values.tolist())


# Functions that are used to reformat and transform the raw
# source files into the master sheet formatting
def uniquify_ids(df, id_col):
    """
    Ensures unique values in a specified ID column by appending an index to duplicates.
    Used on sources which already have some sort of unique(ish) ID.

    Args:
        df (pd.DataFrame): The DataFrame containing the IDs.
        id_col (str): The column name with the IDs to uniquify.
    """
    # Count the occurrences of each ID
    id_counts = df[id_col].value_counts()

    # Create a dictionary to track the number of times an ID has been encountered
    id_tracker = {key: 0 for key in id_counts.index if id_counts[key] > 1}

    # Iterate over the DataFrame and modify the IDs as necessary
    for index, row in df.iterrows():
        id_value = row[id_col]
        if id_counts[id_value] > 1:
            df.at[index, id_col] = f"{id_value}_{id_tracker[id_value]}"
            id_tracker[id_value] += 1


def generate_id_column(df, cols):
    """
    Generates a unique ID column by concatenating specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        cols (List[str]): The column names to concatenate for the unique ID.

    """
    id_col = COL_NAMES['Master']['unique id']

    # Capitalize and join the strings, keeping fully capitalized and CamelCase words unchanged
    def process_value(val, col_name):
        if col_name == COL_NAMES['Master']['transaction date']:
            val = val.replace('/', '-')
        words = val.split()
        processed_words = []
        for word in words:
            if word.isupper() or (word[0].isupper() and not word[1:].islower()):  # handle CamelCase
                processed_words.append(word)
            else:
                processed_words.append(word.capitalize())
        return ''.join(processed_words)

    df[id_col] = df.apply(
        lambda row: '_'.join(process_value(str(row[col]), col) for col in cols),
        axis=1
    )


def str_to_float(s):
    """Converts a string to a float."""
    zero_strings = ['', 'nan', 'null', 'na', 'n/a', 'zero']
    if s.lower() in zero_strings:
        return 0.0
    s = s.replace(',', '')
    s = s.replace('$', '')
    s = s.replace(' ', '')
    s = s.strip()
    return float(s)


def convert_to_mmddyyyy(date: str):
    """Converts a date string to MM/DD/YYYY format."""
    try:
        parsed_date = parser.parse(date)
        return parsed_date.strftime('%m/%d/%Y')
    except (parser.ParserError, TypeError, ValueError):
        raise ValueError(f"Invalid date format: {date}")


def format_live_sheet(df, source):
    """
    Formats a DataFrame from a live Google Sheet based on the source type. Much of the reformatting
    has to do with melting the live sheets so that each row is a separate transaction.

    Args:
        df (pd.DataFrame): The DataFrame to format.
        source (str): The source type (e.g., 'DigitalMarketing', 'JSQ', 'Jeremy').

    Returns:
        pd.DataFrame: The formatted DataFrame.
    """
    header_row_idx = LIVE_SHEETS[source]['header row idx'] - 1  # Set to 0-indexed

    # Drop the rows above the header row (inclusive) and reset the index
    df.drop(index=range(header_row_idx), inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Set the new column names to the header row
    df.columns = df.iloc[0]

    # Drop the header row
    df = df[1:].reset_index(drop=True)

    # Drop totals from DigitalMarketing
    if source == 'DigitalMarketing':
        total_idx = df[df[COL_NAMES[source]['property name']].astype(str).str.contains('Total billings')].index
        if not total_idx.empty:
            df.drop(index=range(total_idx[0], len(df)), inplace=True)

    # Drop totals from JSQ
    if source == 'JSQ':
        last_valid_idx = df[COL_NAMES[source]['property name']].last_valid_index()
        # Select rows up to the last valid index and drop the '2024 Total' column in place
        df.drop(df.index[last_valid_idx+1:], inplace=True)
        df.drop(columns=['2024 Total'], inplace=True)

    # Drop totals from Jeremy
    if source == 'Jeremy':
        last_valid_idx = df[COL_NAMES[source]['property name']].last_valid_index()
        # Drop rows after the last valid index
        df.drop(df.index[last_valid_idx + 1:], inplace=True)

    # Convert column names to datetime objects
    date_cols = [col for col in df.columns if col != COL_NAMES[source]['property name']]
    if source == 'Jeremy':
        date_cols.pop(date_cols.index(COL_NAMES[source]['transaction description']))
        date_cols.pop(date_cols.index(COL_NAMES[source]['GL code']))
        date_cols.pop(date_cols.index(COL_NAMES[source]['GL code description']))

    df.columns = [pd.to_datetime(col, format='%m/%d/%y', errors='raise') if col in date_cols else col for col in df.columns]

    # Massage the df into a format where 'Date', 'Property Name', and 'Amount' are columns (and Project for Jeremy)
    if source == 'Jeremy':
        id_vars = [
            COL_NAMES[source]['property name'],
            COL_NAMES[source]['transaction description'],
            COL_NAMES[source]['GL code'],
            COL_NAMES[source]['GL code description']
        ]
    else:
        id_vars = [COL_NAMES[source]['property name']]

    melted_df = pd.melt(
        df,
        id_vars=id_vars,
        var_name=COL_NAMES[source]['transaction date'],
        value_name=COL_NAMES[source]['debit']
    )

    melted_df[COL_NAMES[source]['debit']] = melted_df[COL_NAMES[source]['debit']].astype(str).apply(str_to_float)

    if source != 'Jeremy':
        # Set the day to the last day of the month
        melted_df[COL_NAMES[source]['transaction date']] += pd.offsets.MonthEnd(0)

    # Get the current date
    now = datetime.now()
    current_month_start = now.replace(day=1)
    next_month_start = (current_month_start + pd.offsets.MonthEnd(1))

    # Keep transactions after the cutoff date, and before the start of the current month
    condition = (
        (melted_df[COL_NAMES[source]['transaction date']] >= LIVE_SHEET_DATE_CUTOFF) &
        (melted_df[COL_NAMES[source]['transaction date']] < current_month_start)
    )

    # Identify the indices of rows to drop
    rows_to_drop = melted_df.index[~condition]

    # Identify rows in the current month
    current_month_condition = (
        (melted_df[COL_NAMES[source]['transaction date']] >= current_month_start)
    )
    current_month_rows = melted_df.loc[current_month_condition]

    # Filter out rows with the "debit" column as zero
    current_month_rows = current_month_rows[current_month_rows[COL_NAMES[source]['debit']] != 0]

    # Print the (nonzero) rows in the current month that are about to be omitted
    if not current_month_rows.empty:
        print("Rows in the current month that are being omitted:")
        print(current_month_rows)

    # Drop the rows in place
    melted_df.drop(rows_to_drop, inplace=True)

    if source == 'Jeremy':
        return melted_df

    # Group by poperty name and date and sum the amounts (except for Jeremy)
    grouped_df = melted_df.groupby(
        [COL_NAMES[source]['property name'], COL_NAMES[source]['transaction date']],
        as_index=False
    ).sum()

    # Set GL Codes (except for Jeremy)
    grouped_df[COL_NAMES[source]['GL code']] = LIVE_SHEETS[source]['GL code']
    grouped_df[COL_NAMES[source]['GL code description']] = LIVE_SHEETS[source]['GL code description']

    return grouped_df


def general_formatting(df, source):
    """
    Some raw columns are exactly what we need, so we can just port those over exactly as they are.
    Other columns receive the same general transformation, regardless of source. The below code
    implements this. However, many raw columns need some sort of manipulation, and these are handled
    in the specific source_to_master functions.

    This function creates many new columns, all w/ master sheet titles.

    Args:
        df (pd.DataFrame): The DataFrame to format.
        source (str): The source type.

    Raises:
        KeyError: If a source column is not found in the DataFrame.

    Returns:
        pd.DataFrame: The formatted DataFrame.
    """
    for col_var, source_col in COL_NAMES[source].items():
        if source_col in df.columns:
            df[COL_NAMES['Master'][col_var]] = df[source_col]
        else:
            raise KeyError(f"Source column {source_col} not found in dataframe.")

    df[COL_NAMES['Master']['source']] = source
    df[COL_NAMES['Master']['QB GL code description']] = df[COL_NAMES['Master']['GL code']].map(GL_CODE_TO_QB_DICT)
    df[COL_NAMES['Master']['transaction date']] = df[COL_NAMES['Master']['transaction date']].astype(str).apply(convert_to_mmddyyyy)
    df[COL_NAMES['Master']['corporate account']] = df[COL_NAMES['Master']['GL code']].map(property_code_to_corporate_acc)
    df[COL_NAMES['Master']['uploaded to saasant as expense?']] = False
    df[COL_NAMES['Master']['uploaded to saasant as invoice?']] = False
    df[COL_NAMES['Master']['uploaded to saasant as bill?']] = False

    now = datetime.now()
    date_string = now.strftime("%m/%d/%Y %H:%M:%S")
    df[COL_NAMES['Master']['date pushed']] = date_string

    for master_col in [COL_NAMES['Master']['debit'], COL_NAMES['Master']['credit']]:
        if master_col in df.columns:
            df[master_col] = df[master_col].astype(str).apply(str_to_float)
        else:
            df[master_col] = 0.0

    # Remove transactions where Debit = Credit = 0
    rows_to_drop = df.index[(df[COL_NAMES['Master']['debit']] == 0.0) & (df[COL_NAMES['Master']['credit']] == 0.0)]
    df.drop(rows_to_drop, inplace=True)

    # Create Unique IDs
    # For live sheets, create a new unique ID
    if source in LIVE_SHEETS.keys():
        # Columns to be concatenated
        id_cols = [
            COL_NAMES['Master']['source'],
            COL_NAMES['Master']['property name'],
            COL_NAMES['Master']['transaction date']
        ]
        # Add project description for Jeremy
        if source == 'Jeremy':
            id_cols.insert(2, COL_NAMES['Master']['transaction description'])

        generate_id_column(df, id_cols)

    # Otherwise, uniquify existing ID
    else:
        # Convert IDs to strings
        df[COL_NAMES['Master']['unique id']] = df[COL_NAMES['Master']['unique id']].astype(str)
        # Run uniquify_ids function
        uniquify_ids(df, COL_NAMES['Master']['unique id'])

    return df


def ramp_to_master(df):
    """Implements Ramp-specific data transformations"""
    df[COL_NAMES['Master']['user']] = df[COL_NAMES['Master']['user']].apply(lambda x: x.split(" - ")[0])


def ramp_reimbursements_to_master(df):
    """Implements Ramp reimbursement-specific data transformations"""
    df[COL_NAMES['Master']['unique id']] = df[COL_NAMES['Master']['unique id']].apply(lambda x: x.split('/')[-1])
    # Rerun uniquify_ids function
    uniquify_ids(df, COL_NAMES['Master']['unique id'])


def amazon_to_master(df):
    """Implements Amazon-specific data transformations"""
    df[COL_NAMES['Master']['vendor']] = 'Amazon'


def old_lowes_to_master(df):
    """Implements Old Lowe's-specific data transformations"""
    df[COL_NAMES['Master']['memo']] = df[COL_NAMES['Master']['memo']].astype(str)
    df[COL_NAMES['Master']['vendor']] = 'Lowes'
    df[COL_NAMES['Master']['SKU']] = df[COL_NAMES['Master']['SKU']].apply(lambda x: x.split(':')[-1])


def qlr_to_master(df):
    """Implements QLR-specific data transformations"""
    df[COL_NAMES['Master']['transaction description']] = "QLR Billback"


def digital_marketing_to_master(df):
    """Implements Digital Marketing-specific data transformations"""
    df[COL_NAMES['Master']['transaction description']] = "DigitalMarketing Billback"


def jsq_to_master(df):
    """Implements JSQ-specific data transformations"""
    df[COL_NAMES['Master']['transaction description']] = "JSQ Billback"


def jeremy_to_master(df):
    """Implements Jeremy-specific data transformations"""
    df[COL_NAMES['Master']['transaction description']] = "Jeremy Billback"
    df[COL_NAMES['Master']['debit']] *= JEREMY_HOURLY_RATE


def delta_amex_to_master(df):
    """Implements Delta Amex-specific data transformations"""
    # source = Amex
    return df


def source_to_master(df, source):
    """Applies transformations to the DataFrame based on the source."""
    df_transformations = {
        'Ramp': ramp_to_master,
        'RampR': ramp_reimbursements_to_master,
        'Amazon': amazon_to_master,
        'OldLowes': old_lowes_to_master,
        'QLR': qlr_to_master,
        'DigitalMarketing': digital_marketing_to_master,
        'JSQ': jsq_to_master,
        'Jeremy': jeremy_to_master,
        'DeltaAmex': delta_amex_to_master
    }

    if source in df_transformations:
        return df_transformations[source](df)
    else:
        raise KeyError(f"Unknown source: {source}")


# Functions that catch mistakes, raise errors, recognize missing data,
# and implement safety features more generally
def get_missing_raw_cols(df, source):
    """
    Identifies columns expected in the DataFrame based on the source but
    that are missing from the actual DataFrame.
    """
    wanted_cols = set(COL_NAMES[source].values())
    missing_cols = wanted_cols - set(df.columns)

    return sorted(missing_cols)


def get_missing_saasant_data(df, source):
    """
    Identifies missing necessary data for SaasAnt export based on the source type.

    Args:
        df (pd.DataFrame): The DataFrame to check.
        source (str): The source type.

    Returns:
        dict: A dictionary where keys are column names and values are lists of row indices
              (1-based) where data is missing.
    """
    necessary_saasant_cols = [
        COL_NAMES['Master']['property name'],
        COL_NAMES['Master']['QB GL code description'],
        COL_NAMES['Master']['transaction date'],
        COL_NAMES['Master']['debit'],
        COL_NAMES['Master']['credit'],
    ]

    if source == 'Amazon':
        necessary_saasant_cols.append(COL_NAMES['Master']['Amazon payment ref id'])

    missing_data = {}
    for col in necessary_saasant_cols:
        empty_value_indices = df[(df[col] == "") | (df[col].isnull())].index.tolist()
        if empty_value_indices:
            missing_data[col] = empty_value_indices
    for key, val in missing_data.items():
        missing_data[key] = [x + 2 for x in val]
    return missing_data


@retry_on_quota_exceeded()
def get_duplicates(df, dst_sheet_list):
    """
    Identifies duplicate unique IDs in the DataFrame compared to a list of destination sheets.

    Args:
        df (pd.DataFrame): The DataFrame to check for duplicates.
        dst_sheet_list (list): A list of gspread worksheets to check against.

    Returns:
        list: A sorted list of duplicate unique IDs.
    """
    id_col_name = COL_NAMES['Master']['unique id']
    existing_unique_ids = set()

    for dst_sheet in dst_sheet_list:
        # Convert all values to strings before updating the set
        col_values = [str(val) for val in dst_sheet.col_values(dst_sheet.find(id_col_name).col)[1:]]
        existing_unique_ids.update(col_values)

    duplicates = existing_unique_ids.intersection(set(df[id_col_name]))

    return sorted(duplicates)


@retry_on_quota_exceeded()
def get_differing_transactions(src_df, dst_sheet_list):
    """
    Identifies duplicate transactions in the source DataFrame (by unique id) that differ
    from those in the destination sheets (by their debit/credit amounts).

    Args:
        src_df (pd.DataFrame): The source DataFrame.
        dst_sheet_list (list): A list of gspread worksheets to compare against.

    Returns:
        pd.DataFrame: A DataFrame of transactions where debit or credit amounts differ.
    """
    id_col = COL_NAMES['Master']['unique id']
    debit_col = COL_NAMES['Master']['debit']
    credit_col = COL_NAMES['Master']['credit']

    refined_src_df = src_df[[id_col, debit_col, credit_col]]

    differing_transactions_df = pd.DataFrame()

    for dst_sheet in dst_sheet_list:
        dst_df = get_cols_by_name_from_sheet(
            dst_sheet,
            [id_col, debit_col, credit_col],
            header_row=1
        )
        dst_df[id_col] = dst_df[id_col].astype(str)
        dst_df[debit_col] = dst_df[debit_col].astype(float)
        dst_df[credit_col] = dst_df[credit_col].astype(float)

        # Merge source and destination DataFrames on the unique ID column
        merged_df = pd.merge(refined_src_df, dst_df, on=id_col, suffixes=('_src', '_dst'))

        # Identify rows where debit or credit amounts differ
        diff_debit = merged_df[merged_df[f'{debit_col}_src'] != merged_df[f'{debit_col}_dst']]
        diff_credit = merged_df[merged_df[f'{credit_col}_src'] != merged_df[f'{credit_col}_dst']]

        # Combine the differing rows
        diff_rows = pd.concat([diff_debit, diff_credit]).drop_duplicates()

        # Append to the final DataFrame
        diff_transactions_df = pd.concat([differing_transactions_df, diff_rows])

    return diff_transactions_df


@retry_on_quota_exceeded()
def problem_with_master_cols(billable_dst_sheet, non_billable_dst_sheet):
    """
    Checks if the columns of the billable and non-billable master sheets match
    and if they match the columns in the code dictionary.
    Unlike previous error-checking functions, this function prints its own error messages.

    Args:
        billable_dst_sheet (gspread.models.Worksheet): The billable destination worksheet.
        non_billable_dst_sheet (gspread.models.Worksheet): The non-billable destination worksheet.

    Returns:
        bool: True if there is a problem with the columns, False otherwise.
    """
    billable_headers = set(billable_dst_sheet.row_values(1))
    non_billable_headers = set(non_billable_dst_sheet.row_values(1))

    if billable_headers != non_billable_headers:
        print(f"❌ Billable and non-billable master sheet columns don't match. Terminating push.")
        print(f"Billable headers:     {sorted(billable_headers)}")
        print(f"Non-billable headers: {sorted(non_billable_headers)}")
        return True

    code_master_cols = set(COL_NAMES['Master'].values())
    actual_master_cols = billable_headers

    if code_master_cols != actual_master_cols:
        print(f"❌ Master sheet columns don't match those in code dictionary. Terminating push.")
        print(f"Code master columns:   {sorted(code_master_cols)}")
        print(f"Actual master columns: {sorted(actual_master_cols)}")
        return True

    return False


if __name__ == '__main__':
    # Get Corporate <> Property CoA Mapping sheet
    coa_map_df = df_from_spread_url(
        CORPORATE_PROPERTY_COA_MAPPING_SHEET['url'],
        sheet_title=CORPORATE_PROPERTY_COA_MAPPING_SHEET['sheet title'])

    # Create mapping from Property GL codes to corporate accounts
    property_code_to_corporate_acc = pd.Series(
        coa_map_df['Corporate Description / Chart of Accounts on QBO'].values,
        index=coa_map_df['Property GL Code / SKU on QBO']
        ).to_dict()
    property_code_to_corporate_acc['N/A'] = 'N/A'

    # Gets urls and names of all spreadsheets in "Ready to Upload to Master Sheet"
    src_files_info = get_files_info_from_folder(READY_TO_UPLOAD_TO_MASTER_SHEET_FOLDER_URL)

    pushed_sources = []
    for src_file_info in src_files_info:
        src_filename = src_file_info['name']
        src_filetype = src_file_info['type']
        src_url = src_file_info['url']
        source = src_filename.split('_')[0]

        # Ignore Google Colab file
        if src_filetype == 'application/vnd.google.colaboratory':
            continue

        print(f"\n⏳Processing {src_filename}...")

        df = df_from_file_url(src_url, src_filetype)

        # Get live sheets into a more readable and standard format
        if source in LIVE_SHEETS.keys():
            # We need to create a new df here bc of the melting and grouping
            df = format_live_sheet(df, source)

        # Check for missing columns in the raw spreadsheet/csv
        missing_col_names = get_missing_raw_cols(df, source)
        if len(missing_col_names) > 0:
            print(f"❌ Could not push {src_filename}: missing {source} column(s).")
            print(f"Missing columns: {[col_name for col_name in missing_col_names]}")
            continue

        # Refine the dataframe to just the relevant columns and relable them w/ master sheet column
        # names. Add default columns like "Uploaded to SaasAnt as Bill" = False or the Source column
        # (this is a mutator function)
        general_formatting(df, source)

        # Apply transformations to finalize the upload-ready df.
        # Returns a dict w/ two dfs: billable and non-billable
        source_to_master(df, source)

        # Open the destination sheets
        billable_dst_sheet = open_sheet_from_url(
            MASTER_SHEET['url'],
            sheet_title=MASTER_SHEET['sheet title']
        )
        non_billable_dst_sheet = open_sheet_from_url(
            NON_BILLABLE_SHEET['url'],
            sheet_title=NON_BILLABLE_SHEET['sheet title']
        )

        # Check that the column names match between billable and non-bllable master sheets and are
        # the same as the global COL_NAMES dictionary
        if problem_with_master_cols(billable_dst_sheet, non_billable_dst_sheet):
            # The error messages are printed inside the function
            break

        # Check for missing mandatory SaasAnt data
        missing_saasant_data = get_missing_saasant_data(df, source)
        if len(missing_saasant_data.keys()) > 0:
            print(f"❌ Could not push {src_filename}: missing mandatory SaasAnt data.")
            print(f"Missing data: {[item for item in missing_saasant_data.items()]}")
            continue

        # Check for duplicates
        duplicates = get_duplicates(df, [billable_dst_sheet, non_billable_dst_sheet])

        if source in LIVE_SHEETS.keys():
            # Check that previously pushed transactions still match the live sheet
            differing_transactions = get_differing_transactions(
                df,
                [billable_dst_sheet, non_billable_dst_sheet]
            )
            if not differing_transactions.empty:
                print(f"🛑 WARNING - Could not push {src_filename}: found previously pushed "
                      "transactions in the master sheet that no longer match their source sheet.")
                print(f"Differing transactions:\n{differing_transactions}")
                continue
            # Only upload new transactions from Marketing Billbacks (remove rows with duplicate IDs)
            rows_to_drop = df.index[df[COL_NAMES['Master']['unique id']].isin(duplicates)].tolist()
            df.drop(rows_to_drop, inplace=True)
            print(f"{src_filename} has {df.shape[0]} new transactions that will be pushed")
        elif duplicates:
            print(f"❌ Could not push {src_filename}: duplicates found.")
            print(f"Duplicate IDs: {duplicates}")
            continue

        # Return billback and non-billback dfs
        billable_filter = (df[COL_NAMES['Master']['property name']].isin(PROPERTY_NAMES))
        master_billbacks_df = df[billable_filter]
        master_non_billbacks_df = df[~billable_filter]

        # Push to master sheets
        append_df_to_sheet(master_billbacks_df, billable_dst_sheet)
        append_df_to_sheet(master_non_billbacks_df, non_billable_dst_sheet)
        print(f"✅ Pushed {src_filename}.")
        pushed_sources.append(source)

        if RELOCATE_SPREAD_MODE:
            move_file(src_url, ARCHIVED_CODED_FILES_FOLDER_URL)

    pushed_sources.sort()
    total_sources = sorted(source for source in COL_NAMES.keys())
    total_sources.remove('Master')
    print("\n")
    print(f"Sources pushed: "
          f"{', '.join(src if src in pushed_sources else '_'*len(src) for src in total_sources)}")
    print(f"Total sources:  {', '.join(total_sources)}")


KeyboardInterrupt: 